# Extracting entities given the dataset with help of LLM prompt
Entity Extraction with Generative Models


In [1]:
# importing libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
# reading the data
df = pd.read_json('../data/relations_dev.txt')

In [39]:
#df_nested_list.head()

In [91]:
import json
# load data using Python JSON module
with open('../data/relations_dev.txt','r') as f:
    data = json.loads(f.read())
# Flatten data
df_nested_list2 = pd.json_normalize(data, record_path =['relations'])

In [19]:
concatenated.head()

,text,start,end,token_start,token_end,entityLabel,child,head,relationLabel
0,Bachelor,0,8,0,0,DIPLOMA,4.0,0.0,DEGREE_IN
1,Mechanical Engineering,21,43,4,5,DIPLOMA_MAJOR,7.0,0.0,DEGREE_IN
2,Physical Science,47,63,7,8,DIPLOMA_MAJOR,15.0,9.0,EXPERIENCE_IN
3,3+ years,64,72,9,11,EXPERIENCE,18.0,9.0,EXPERIENCE_IN
4,developing,89,99,15,15,SKILLS,22.0,9.0,EXPERIENCE_IN


In [25]:
#clean_df = pd.concat([df1, concatenated], axis="columns")

In [41]:
#clean_df.head()

In [28]:
clean_df.shape

(145, 10)

In [29]:
#clean_df.to_csv('../data/relation.csv')

In [40]:
#clean_df.head()

In [32]:
# installing the libe
!pip install cohere requests tqdm

In [92]:
import cohere
import pandas as pd
import requests
import datetime
from tqdm import tqdm
pd.set_option('display.max_colwidth', None)

In [93]:
def get_post_titles():
    data = pd.read_json('../data/relations_dev.txt')
    return data['document']

In [42]:
# API
my_API = "nMqcDQ2gC0vxFVwIDvUINps6zJTvYZOskiz5dhq1"

In [94]:
# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(my_API)

## Preparing examples for the prompt
In our prompt, we'll present the model with examples for the type of output we're after. We basically get a set of subreddit article titles, and label them ourselves. The label here is the name of the movie mentioned in the title (and "none" if no movie is mentioned).

In [115]:
 # job description
job_examples =[("text", "Bachelor",
        "start", '0',
        "end", '8',
        "token_start", '0',
        "token_end", '0',
        "entityLabel", "DIPLOMA"),  
        ("text", "Mechanical Engineering",
        "start", '21',
        "end", '43',
        "token_start", '4',
        "token_end", '5',
        "entityLabel", "DIPLOMA_MAJOR"), 
        ( "text", "10+ years",
        "start", '0',
        "end", '9',
        "token_start", '0',
        "token_end", '2',
        "entityLabel", "EXPERIENCE"),
        ("text", "8+ years",
        "start", '0',
        "end", '8',
        "token_start", '0',
        "token_end", '2',
        "entityLabel", "EXPERIENCE")]

### Creating the extraction prompt

In [111]:
class cohereExtractor():
    def __init__(self, examples, example_labels, labels, task_desciption, example_prompt):
        self.examples = examples
        self.example_labels = example_labels
        self.labels = labels
        self.task_desciption = task_desciption
        self.example_prompt = example_prompt

    def make_prompt(self, example):
        examples = self.examples + [example]
        labels = self.example_labels + [""]
        return (self.task_desciption +
                "\n---\n".join( [examples[i] + "\n" +
                                self.example_prompt + 
                                 labels[i] for i in range(len(examples))]))

    def extract(self, example):
        extraction = co.generate(
          model='large',
          prompt=self.make_prompt(example),
          max_tokens=10,
          temperature=0.1,
          stop_sequences=["\n"])
        return(extraction.generations[0].text[:-1])


cohereJobExtractor = cohereExtractor([e[1] for e in job_examples], 
                                       [e[0] for e in job_examples], [],
                                       "", 
                                       "the entites:")

# Uncomment to inspect the full prompt:
# print(cohereMovieExtractor.make_prompt('<input text here>'))

In [114]:
# This is what the prompt looks like:
print(cohereJobExtractor.make_prompt("text"))

Bachelor
the entites:text
---
Mechanical Engineering
the entites:text
---
10+ years
the entites:text
---
8+ years
the entites:text
---
starts
the entites:


## Getting the data
Make the API call to get the jobs.

In [90]:
num_posts = 10
job_list = get_post_titles(size = num_posts, 
      after=str(int(datetime.datetime(2021,1,1,0,0).timestamp())), 
      before=str(int(datetime.datetime(2022,1,1,0,0).timestamp())), 
      subreddit="jobs", 
      sort_type="score", 
      sort="desc")

# Show the list
job_list

TypeError: get_post_titles() got an unexpected keyword argument 'size'

## Running the model
And now we loop over the posts and process each one of them with our extractor.

In [79]:
results = []
for text in tqdm(job_list):
    try:
        extracted_text = cohereMovieExtractor.extract(text)
        results.append(extracted_text)
    except Exception as e:
        print('ERROR: ', e)

100%|██████████| 10/10 [00:12<00:00,  1.30s/it]


In [81]:
# look at the result
pd.DataFrame(data={'text': job_list, 'extracted_entity': results})

,text,extracted_entity
0,Boss actually tried demanding me to donate to the companies “charity”,B
1,Imposter Syndrome,B
2,Oh no. How do I say “I enjoyed not having a biweekly mental breakdown over work and school” professionally?,B
3,3 minutes into the interview the interviewer goes on a tangent about how I'm not a fit for the role.,B
4,"My dad, with 35 years experience as an engineer, just discovered the bullshit that is today's application process.",B
5,How to help parents understand that it’s difficult to get hired these days?,B
6,Intentionally bombed an interview and had fun doing it,B
7,Interviewers not respecting potential hires,B
8,I got an offer!! Such a great feeling when a company finally sees your worth!,B
9,"People don't leave bad jobs, they leave bad bosses.",B


## How well does this work?
We can better measure the performance of this extraction method using a larger labeled dataset. So let's load a test set of 100 examples:

In [ ]:
test_df = pd.read_csv('https://raw.githubusercontent.com/cohere-ai/notebooks/main/notebooks/data/movie_extraction_test_set_100.csv',index_col=0)
test_df

Let's run the extractor on these post titles (calling the API in parallel for quicker results):

In [ ]:
from concurrent.futures import ThreadPoolExecutor

extracted = []
# Run the model to extract the entities
with ThreadPoolExecutor(max_workers=8) as executor:
    for i in executor.map(cohereMovieExtractor.extract, test_df['text']):
        extracted.append(str(i).strip())
# Save results
test_df['extracted_text'] = extracted

In [ ]:
#Let's look at some results:

test_df.head()

Let's calculate the accuracy by comparing to the labeled examples

In [ ]:
# Compare the label to the extracted text
test_df['correct'] = (test_df['label'].str.lower() == test_df['extracted_text'].str.lower()).astype(int)

# Print the accuracy
print(f'Classification accuracy {test_df["correct"].mean() *100}%')

So it seems this prompt works well on this small test set. It's not guaranteed it will do as well on other sets, however. The prompt can be improved by trying on more data, discovering edge cases, and adding more examples to the prompt.

We can look at the examples it got wrong: